In [1]:
# VaR And Options

from scipy.stats import multivariate_normal as mvn
from scipy.stats import norm
from scipy.stats import chi2

import numpy as np
import pandas as pd
from scipy import optimize
import seaborn as sns
import math

In [2]:
def myPriceEuroOption(S, K, std, q, rf, T, ty):
    _d1u = math.log(S / K) + (rf - q + std ** 2 / 2) * T
    _d1d = std * math.sqrt(T)
    d1 = _d1u / _d1d
    d2 = d1 - _d1d

    if ty == "call":
        p = S * math.exp(-q * T) * norm.cdf(d1) - K * math.exp(-rf * T) * norm.cdf(d2)
        delta = math.exp(-q * T) * norm.cdf(d1)
    if ty == "put":
        p = K * math.exp(-rf * T) * norm.cdf(-d2) - S * math.exp(-q * T) * norm.cdf(-d1)
        delta = -math.exp(-q * T) * norm.cdf(-d1)
    gamma = norm.pdf(d1) / (S * std * math.sqrt(T))
    
    return [p, delta, gamma]

In [3]:
# Option Replication
S = 100 * 1
std = 0.2
rf = 0.1
q = 0
T = 3
otype = "put"

K = 100

z_alpha = norm.ppf(0.05)


[p1, delta1, gamma1] = myPriceEuroOption(S, K, std, q, rf, T, otype)


# =================================================
# Dollar position of replication
re_stock = delta1 * S
re_bond = p1 - delta1 * S

print()
print("position of replicate stock:", round(re_stock, 4))
print("position of replicate bond:", round(re_bond, 4))


position of replicate stock: -14.9349
position of replicate bond: 18.0916


In [4]:
# One Option, Delta VaR
S = 100
std = 0.198
rf = 0.0995
q = 0
T = 1
otype = "call"
K = 100

mu_annu = 0.2

z_alpha = norm.ppf(0.05)



[c1, delta1, gamma1] = myPriceEuroOption(S, K, std, q, rf, T, otype)


# Delta VaR, daily
exp_delta = (c1 - delta1 * S) * rf / 252 \
            + delta1 * S * mu_annu / 252
std_delta = math.sqrt((delta1 * S) ** 2 * std ** 2 / 252)

VaR_delta = - (exp_delta + z_alpha * std_delta)
VaR_delta_s = - (0 + z_alpha * std_delta)

print("daily Delta VaR:", round(VaR_delta, 4))
print("daily Delta VaR, simplified mu:", round(VaR_delta_s, 4))


# Delta VaR, monthly
exp_delta = (c1 - delta1 * S) * rf / 12 \
            + delta1 * S * mu_annu / 12
std_delta = math.sqrt((delta1 * S) ** 2 * std ** 2 / 12)

VaR_delta = - (exp_delta + z_alpha * std_delta)
VaR_delta_s = - (0 + z_alpha * std_delta)

print("monthly Delta VaR:", round(VaR_delta, 4))
print("monthly Delta VaR, simplified mu:", round(VaR_delta_s, 4))

daily Delta VaR: 1.4558
daily Delta VaR, simplified mu: 1.49
monthly Delta VaR: 6.1105
monthly Delta VaR, simplified mu: 6.828


In [5]:
# Two Options, Delta VaR
S1 = 100
S2 = 200

rf = 6 / 100
q = 0

T1 = 1
T2 = T1

std1 = 0.15
std2 = 0.3
rho_12 = -0.3

K1 = S1
K2 = S2

otype1 = "call"
otype2 = "call"

mu_annu = 0
z_alpha = norm.ppf(0.05)



[c1, delta1, gamma1] = myPriceEuroOption(S1, K1, std1, q, rf, T1, otype1)
[c2, delta2, gamma2] = myPriceEuroOption(S2, K2, std2, q, rf, T2, otype2)

init_val_port = c1 + c2    # initial value of portfolio

# calculate the portfolio shares of S1, S2
w1 = delta1 * S1 / init_val_port
w2 = delta2 * S2 / init_val_port
w_B = 1 - (w1 + w2)     # shares of lending

# annual variance of return
var_port = (w1 * std1) ** 2 + (w2 * std2) ** 2 \
            + 2 * rho_12 * w1 * std1 * w2 * std2


#1-day VaR on portfolio
VaR_port = - (0 + z_alpha * math.sqrt(var_port / 252) * init_val_port)
print("daily Delta VaR at 0.005:", round(VaR_port, 4))


#1-month VaR on portfolio
VaR_port = - (0 + z_alpha * math.sqrt(var_port / 12) * init_val_port)
print("montly Delta VaR at 0.005:", round(VaR_port, 4))

daily Delta VaR at 0.005: 3.7789
montly Delta VaR at 0.005: 17.3172


In [8]:
# Straddle, Delta Gamma VaR
S = 18759
std = 0.15
rf = 0.04
q = 0
T = 0.25

K = 19000

otype1 = "call"
otype2 = "call" 

# here w stands for the unit of position.
w1 = -1
w2 = -1

mu_annu = 0
z_alpha = norm.ppf(0.05)



[c1, delta1, gamma1] = myPriceEuroOption(S, K, std, q, rf, T, otype1)
[c2, delta2, gamma2] = myPriceEuroOption(S, K, std, q, rf, T, otype2)

# calculate the portfolio of options
# this is a short position
opt_initial = np.dot([w1, w2], [c1, c2])
delta_port = np.dot([w1, w2], [delta1, delta2])
gamma_port = np.dot([w1, w2], [gamma1, gamma2])

E_delta1 = delta1 * S * mu_annu + (c1 - delta1 * S) * rf
E_delta2 = delta2 * S * mu_annu + (c2 - delta2 * S) * rf
E_delta_port = E_delta1 * w1 + E_delta2 * w2


# month Delta-Gamma VaR
month_std = std / math.sqrt(12)
var_stock = (S * std) ** 2 / 12

DVaR_port = - (0 + z_alpha * abs(delta_port) * S * month_std)
DVaR_port_m = - (E_delta_port / 12 + z_alpha * abs(delta_port) * S * month_std)
DGVaR_port = - (0 + z_alpha * abs(delta_port) * S * month_std\
             + (z_alpha ** 2) / 2 * gamma_port * var_stock)
DGVaR_port_m = - (E_delta_port / 12 + z_alpha * abs(delta_port) * S * month_std\
             + (z_alpha ** 2) / 2 * gamma_port * var_stock)
print("monthly Delta VaR, simplified:", round(DVaR_port, 4))
print("monthly Delta VaR, with mu:", round(DVaR_port_m, 4))
print("monthly Delta-Gamma VaR, simplified:", round(DGVaR_port, 4))
print("monthly Delta-Gamma VaR, with mu:", round(DGVaR_port_m, 4))
print()


# daily Delta-Gamma VaR
day_std = std / math.sqrt(252)
var_stock = (S * std) ** 2 / 252

DVaR_port = - (0 + z_alpha * abs(delta_port) * S * day_std)
DVaR_port_m = - (E_delta_port / 252 + z_alpha * abs(delta_port) * S * day_std)
DGVaR_port = - (0 + z_alpha * abs(delta_port) * S * day_std\
             + (z_alpha ** 2) / 2 * gamma_port * var_stock)
DGVaR_port_m = - (E_delta_port / 252 + z_alpha * abs(delta_port) * S * day_std\
             + (z_alpha ** 2) / 2 * gamma_port * var_stock)
print("daily Delta VaR, simplified:", round(DVaR_port, 4))
print("daily Delta VaR, with mu:", round(DVaR_port_m, 4))
print("daily Delta-Gamma VaR, simplified:", round(DGVaR_port, 4))
print("daily Delta-Gamma VaR, with mu:", round(DGVaR_port_m, 4))

monthly Delta VaR, simplified: 1336.766
monthly Delta VaR, with mu: 1277.7802
monthly Delta-Gamma VaR, simplified: 1842.9568
monthly Delta-Gamma VaR, with mu: 1783.971

daily Delta VaR, simplified: 291.7063
daily Delta VaR, with mu: 288.8974
daily Delta-Gamma VaR, simplified: 315.8106
daily Delta-Gamma VaR, with mu: 313.0017
